In [ ]:
import json

# Access parameters directly
bronze_output = ""

In [ ]:
# Manual Try

#from datetime import date, timedelta

# For getting the data manually without data factory pipeline
#start_date = date.today() - timedelta(days=1)

#bronze_adls = "abfss://bronze@tut03datalake.dfs.core.windows.net/"
#silver_adls = "abfss://silver@tut03datalake.dfs.core.windows.net/"

In [ ]:
# Automated process of the above code cell.

# Parse the JSON string
output_data = json.loads(bronze_output)

# Access Individual Variables
start_date = output_data.get("start_date")
silver_adls = output_data.get("silver_adls")
bronze_adls = output_data.get("bronze_adls")

print(f"Start Date: {start_date}")
print(f"Silver ADLS: {silver_adls}")
print(f"Bronze ADLS: {bronze_adls}")

In [ ]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [ ]:
# Load the JSON data into a Spark Datafrme
df = spark.read.option("multiline", "true").json(f"{bronze_adls}/{start_date}_earthquake_data.json")

In [ ]:
df.head()

In [ ]:
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated'),
    )
)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
# Validating the data> Checking for missing values 
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [ ]:
# Updating the time from time=1747438679005, updated=1747454278106 TO
# time=datetime.datetime(2025, 5, 16, 23, 37, 59, 5000), updated=datetime.datetime(2025, 5, 17, 3, 57, 58, 106000)
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [ ]:
df.head()

In [ ]:
# Saving the transformed dataframe to the SILVER container
silver_data = f"{silver_adls}earthquake_event_silver/"

In [ ]:
# Append DF to Silver container in Parquet format
df.write.mode('append').parquet(silver_data)

In [ ]:
mssparkutils.notebook.exit(silver_data)